# Simulate Battery for Solar Plant

## Introduction

Notebook to simulate batteries of different sizes for an existing solar plant. 

On sunny days your PV system usually feeds the excess energy that you cannot use at home to the grid, while you have to draw energy from the grid on cloudy days (and at night). Your own consumption can be increased by adding a battery to your PV system. The size of the battery (and inverter) heavily depends on the setup of you PV system and epecially on your personal power consumption profile.

Modern energy meters provide all the information needed to simulate different batteries and inverters for your personal needs. This software uses this data to find out which combination best fits your needs and assists you in your buying decision. The script simulates batteries of different sizes which are charged, when power is fed to the grid and discharged, when power is drawn from the grid.

## Inverter and battery settings

In [ ]:
# Maximum charging and discharging power of battery / inverter combination in W
max_discharging_power = 5400
max_charging_power = 5400

# Charge and discharge efficiency
discharge_efficiency = 0.9 # 90% efficiency by discharging
charge_efficiency = 0.9 # 90% efficiency by charging

# Set sizes (in kWh) and names of batteries of to be simulated
battery_sizes = [4.56, 15.7, 31.4]
battery_names = [
    "5 kWh - 1x PylonTech US5000C",  
    "16 kWh DHZ",  
    "32 kWh DHZ"
]

## Import and load data

In [ ]:
import argparse
import datetime
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Location CSV
csv_file_path = 'C:/Users/export_test.csv'

# Load data
all_data = pd.read_csv(csv_file_path)
df = pd.DataFrame.from_records(all_data, columns = ['timestamp', 'power'])
df['time'] = pd.to_datetime(df['timestamp'], format='ISO8601')
all_data.head()

## Calculate inverter limitations

In [ ]:
# Excess demand (Power demand exceeds what the inverter can deliver)
df_overload = df[df['power'] > max_discharging_power].copy()
df_overload['excess_wh'] = (df_overload['power'] - max_discharging_power) / 60  # 1-minuut interval
df_overload['month'] = df_overload['time'].dt.to_period('M')
monthly_excess_demand = df_overload.groupby('month')['excess_wh'].sum() / 1000  # to kWh

# Excess PV Input (More PV power than the inverter can charge)
df_overpv = df[df['power'] < -max_charging_power].copy()
df_overpv['excess_pv_wh'] = (-df_overpv['power'] - max_charging_power) / 60
df_overpv['month'] = df_overpv['time'].dt.to_period('M')
monthly_excess_pv = df_overpv.groupby('month')['excess_pv_wh'].sum() / 1000  # to kWh

# Combine values into a table
df_demand = monthly_excess_demand.reset_index().rename(columns={'excess_wh': 'excess_demand_kwh'})
df_pv = monthly_excess_pv.reset_index().rename(columns={'excess_pv_wh': 'excess_pv_kwh'})

# Merge by month
monthly_combined = pd.merge(df_demand, df_pv, on='month', how='outer').fillna(0)

# Add total row
total_row = pd.DataFrame({
    'month': ['TOTAAL'],
    'excess_demand_kwh': [monthly_combined['excess_demand_kwh'].sum()],
    'excess_pv_kwh': [monthly_combined['excess_pv_kwh'].sum()]
})

monthly_combined = pd.concat([monthly_combined, total_row], ignore_index=True)

# Round to 2 decimal places
monthly_combined[['excess_demand_kwh', 'excess_pv_kwh']] = monthly_combined[['excess_demand_kwh', 'excess_pv_kwh']].round(2)
print("Excess power (demand or PV) due to inverter limitations")
print(monthly_combined)

## Calculate power and energy usable by battery

In [ ]:
# calculate column power_battery. 
# power in this column is limited by max_discharging_power and max_charging_power of the inverter
df['power_battery'] = df['power']
df.loc[df['power'] > max_discharging_power, 'power_battery'] = max_discharging_power
df.loc[df['power'] < -max_charging_power, 'power_battery'] = -max_charging_power

df['power_battery_eff'] = df['power_battery'] 
df.loc[df['power_battery']>0, 'power_battery_eff'] = df['power_battery'] / discharge_efficiency
df.loc[df['power_battery']<0, 'power_battery_eff'] = df['power_battery'] * charge_efficiency


df['power_battery_usable'] = df['power_battery_eff']
df.loc[df['power_battery_usable'] > max_discharging_power, 'power_battery_usable'] = max_discharging_power
df.loc[df['power_battery_usable'] < -max_charging_power, 'power_battery_usable'] = -max_charging_power

# calculate excess power, not usable by battery due to above limitation of min/max (dis)charging power.
# This fraction is always fed or drawn to/from the grid
df['power_excess'] = df['power'] - df['power_battery_usable']

# Calculate time difference to previous row
df['time_period'] = df['time'].diff()
df['time_period'] = df['time_period'] /np.timedelta64(1,'s')

# Calculate Energy usable by battery for charging / discharging in kWh
df['energy_battery'] = - df['time_period'] * df['power_battery_usable'] / (60 * 60 * 1000)
df['energy_battery'] = df['energy_battery'].fillna(0)
df.head()

## Battery simulation

In [ ]:
# Function to simulate a battery of size battery_size. 
# df contains energy that could be charged or discharged at each time.
def simulate_battery(df, battery_size):
    print('Simulating battery of size ', str(battery_size), ' kWh.')
    last_energy_in_battery = 0 
    last_total_energy_provided_by_battery = 0 

    energy_in_battery = []
    energy_excess_to_grid = []
    energy_provided_by_battery = []
    total_energy_provided_by_battery = []

    # go through all rows and check if energy can be charged or discharged to battery
    for row in df.iterrows():
        # Add energy of this row to battery and check if battery is either full or empty. 
        # In this case, set battery state to battery_size or zero. 
        # Copy excess energy, not taken by battery to this_energy_excess_to_grid
        this_energy_in_battery =  row[1]['energy_battery'] + last_energy_in_battery 
        # If battery is full, copy excess to this_energy_excess_to_grid
        if this_energy_in_battery > battery_size:
            this_energy_excess_to_grid = this_energy_in_battery - battery_size
            this_energy_in_battery = battery_size
        # If battery is empty, copy excess to this_energy_excess_to_grid
        elif this_energy_in_battery < 0:
            this_energy_excess_to_grid = this_energy_in_battery
            this_energy_in_battery = 0
        # If battery is not full or empty, set this_energy_excess_to_grid to zero
        else:
            this_energy_excess_to_grid = 0
        
        # Append data to lists
        energy_in_battery.append( this_energy_in_battery )
        energy_excess_to_grid.append(this_energy_excess_to_grid)
            
            
        # Check if the battery was discharged in this row.
        # If this is the case, copy energy of this row to new column. 
        # And calculate the total of the energy discharged from the battery. 
        # This value is used to see how much energy was used from the battery
        
        if this_energy_in_battery < last_energy_in_battery:
            energy_provided_by_battery.append(last_energy_in_battery - this_energy_in_battery)
            last_total_energy_provided_by_battery = last_total_energy_provided_by_battery + last_energy_in_battery - this_energy_in_battery
            total_energy_provided_by_battery.append(last_total_energy_provided_by_battery)
        # If battery was charged, do not add energy to list
        else:
            energy_provided_by_battery.append(0)
            total_energy_provided_by_battery.append(last_total_energy_provided_by_battery)

        last_energy_in_battery = this_energy_in_battery
        
    return energy_in_battery, energy_excess_to_grid, energy_provided_by_battery, total_energy_provided_by_battery

# Go through all batteries and start simulation for each one. Add results as columns to df.
for i in range(len(battery_sizes)):
    energy_in_battery, energy_excess_to_grid, power_provided_by_battery, total_power_provided_by_battery = simulate_battery(df, battery_sizes[i])
    #battery_charge, battery_charge_excess, battery_discharge, battery_total_discharge = simulate_battery(df, battery_sizes[i])

    df['energy_in_battery_' + battery_names[i]] = energy_in_battery #battery_charge
    df['energy_excess_to_grid_' + battery_names[i]] = energy_excess_to_grid #battery_charge_excess
    df['energy_provided_by_battery_' + battery_names[i]] = power_provided_by_battery #battery_discharge #power_provided_by_battery
    df['total_energy_provided_by_battery_' + battery_names[i]] = total_power_provided_by_battery #battery_total_discharge #total_energy_provided_by_battery
df.tail()

## Battery details and graphs

In [ ]:
# Energy used from grid and battery
bat_energy = []
grid_energy = []
total_energy_inverter_excess = -monthly_excess_demand.sum()
for bat in battery_names:
    bat_energy.append(df['energy_provided_by_battery_' + bat].sum())
    grid_energy.append(-df.loc[df['energy_excess_to_grid_' + bat] < 0, 'energy_excess_to_grid_' + bat].sum() - total_energy_inverter_excess)
total_energy = [bat_energy[i] + grid_energy[i] for i in range(len(bat_energy))] 

#print(bat_energy)
#print(grid_energy)
#print(total_energy)

# Plot energy used from grid and from battery for different battery sizes
# Use Seaborn style
sns.set_style('ticks')
plt.figure(figsize=(7,5))
plt.plot(battery_sizes,total_energy, label = 'Total energy (grid + battery) = Without battery')
plt.plot(battery_sizes,bat_energy, label = 'Energy used from battery')
plt.plot(battery_sizes,grid_energy, label = 'Energy used from grid')

plt.ylim(ymin=0) 
plt.legend()
plt.xlabel("Battery size [kWh]")
plt.ylabel("Energy [kWh]")
plt.title("Energy used from grid and battery by battery size")
plt.show()

# Create list that contains column names of batteries
columns =[]
for bat in battery_names:
    columns.append('total_energy_provided_by_battery_' + bat)#
    
print("Total energy used from battery instead of grid (in kWh):")
# Last line in df contains the total energy (in kWh) that was used from the battery
df.iloc[-1][columns]

# Plot Energy used from battery by battery size
plt.figure(figsize=(9,5))
plt.plot(battery_sizes, df.iloc[-1][columns].tolist(), label = "total_energy_provided_by_battery_" + bat)
plt.xlabel("Battery size [kWh]")
plt.ylabel("Energy used from battery [kWh]")
plt.title("Energy used from battery by battery size")
plt.show()

# Plot total energy used by each battery over time
plt.figure(figsize=(9,5))
for bat in battery_names:
    plt.plot(df['time'],df['total_energy_provided_by_battery_' + bat], label = "total_energy_provided_by_battery_" + bat)
plt.legend()
plt.xlabel("Date")
plt.ylabel("Energy used from battery [kWh]")
plt.title("Energy used from battery over time")
plt.show()

# Plot State of Charge for Batteries 
plt.figure(figsize=(9,5))
for i in range(len(battery_names)):
    Ladestatus = df['energy_in_battery_' + battery_names[i]] # / Batterie_Groessen[i] * 100
    plt.plot(df['time'], Ladestatus, label = 'energy_in_battery_' + battery_names[i])
plt.legend()
plt.xlabel("Date")
plt.title("Charge / discharge curves")
plt.ylabel("Energy stored in battery [kWh]")
plt.show()

# Monthly Details
print("Monthly Details")
# Used timestamp as index and group results by month
df.index = pd.to_datetime(df['timestamp'], format='ISO8601')
df = df.drop(['timestamp', 'time'], axis=1)

Monatswerte = df.groupby(by=[df.index.month, df.index.year]).sum()

# Copy index to new column. Is used afterwards, to create a label of format YYYY-MM
Monatswerte.index = Monatswerte.index.rename(['Month', 'Year'])
Monatswerte = Monatswerte.reset_index(level=[0,1])
Monatswerte['label'] = Monatswerte.Year.astype(str) + '-' + Monatswerte.Month.astype(str)
Monatswerte = Monatswerte.set_index('label')

# Show table containing monthly values
columns =[]
for bat in battery_names:
    columns.append('energy_provided_by_battery_' + bat)#

# Plot monthly values as bar chart
ax = Monatswerte[columns].plot.bar(figsize=(9,5), title='Monthly details')
ax.set_xlabel("Month")
ax.set_ylabel("Energy provided by battery [kWh]")
Monatswerte[columns]

# ToDo:
- check length of time period used to calculate energy. If the sensor was removed, data is missing. This results in a long time period between two measurements. As the energy is calculated by multipling the power with the time period, this can result in a huge step.

- Increase performence by improving simulate_battery(). Try to use vectorization instead of the for loop.

- Consider losses during charge and discharge with efficiency curve.
